<a href="https://colab.research.google.com/github/prakharsrivastava/powerbi/blob/main/Data_Science_on_Google_Cloud_Platform_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/chapter2/


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/chapter2


In [57]:
import pandas as pd
df = pd.read_csv('airports.csv',low_memory=False,nrows=3)
df.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,W,54,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,W,4,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,W,10,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN


In [ ]:
df["LONGITUDE"]

In [11]:
!pip install apache-beam[gcp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.2/526.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 KB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 KB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.

In [16]:
#!/usr/bin/env python3

# Copyright 2016 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import apache_beam as beam
import logging
import csv
import json


def addtimezone(lat, lon):
    try:
        import timezonefinder
        tf = timezonefinder.TimezoneFinder()
        return lat, lon, tf.timezone_at(lng=float(lon), lat=float(lat))
        # return (lat, lon, 'America/Los_Angeles') # FIXME
    except ValueError:
        return lat, lon, 'TIMEZONE'  # header


def as_utc(date, hhmm, tzone):
    try:
        #print(date+" "+hhmm+" "+tzone) #2015-04-28 1600 America/Anchorage
        if len(hhmm) > 0 and tzone is not None:
            import datetime, pytz
            loc_tz = pytz.timezone(tzone)
            loc_dt = loc_tz.localize(datetime.datetime.strptime(date, '%Y-%m-%d'), is_dst=False)
            # can't just parse hhmm because the data contains 2400 and the like ...
            loc_dt += datetime.timedelta(hours=int(hhmm[:2]), minutes=int(hhmm[2:]))
            utc_dt = loc_dt.astimezone(pytz.utc)
            return utc_dt.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return ''  # empty string corresponds to canceled flights
    except ValueError as e:
        logging.exception('{} {} {}'.format(date, hhmm, tzone))
        raise e


def tz_correct(line, airport_timezones):
    print(line)
    print(airport_timezones)
    fields = json.loads(line)
    try:
        # convert all times to UTC
        dep_airport_id = fields["ORIGIN_AIRPORT_SEQ_ID"]
        arr_airport_id = fields["DEST_AIRPORT_SEQ_ID"]
      
        dep_timezone = airport_timezones[dep_airport_id][2]
        arr_timezone = airport_timezones[arr_airport_id][2]

        for f in ["CRS_DEP_TIME", "DEP_TIME", "WHEELS_OFF"]:
            fields[f] = as_utc(fields["FL_DATE"], fields[f], dep_timezone)
        for f in ["WHEELS_ON", "CRS_ARR_TIME", "ARR_TIME"]:
            fields[f] = as_utc(fields["FL_DATE"], fields[f], arr_timezone)

        yield json.dumps(fields)
    except KeyError as e:
        logging.exception(" Ignoring " + line + " because airport is not known")


In [10]:
!pip install TimezoneFinder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:

import apache_beam as beam
import csv
if __name__ == '__main__':
    with beam.Pipeline('DirectRunner') as pipeline:
        airports = (pipeline
                    | 'airports:read' >> beam.io.ReadFromText('airports_10_lines.csv')
                    | beam.Filter(lambda line: "United States" in line)
                    | 'airports:fields' >> beam.Map(lambda line: next(csv.reader([line])))
                    | 'airports:tz' >> beam.Map(lambda fields: (fields[0], addtimezone(fields[21], fields[26])))
                 
                    )

        flights = (pipeline
                   | 'flights:read' >> beam.io.ReadFromText('flights_sample.json')
                   | 'flights:tzcorr' >> beam.FlatMap(tz_correct, beam.pvalue.AsDict(airports))
                   | "print" >> beam.Map(print)
                   )

 

{"FL_DATE":"2015-04-28","UNIQUE_CARRIER":"EV","ORIGIN_AIRPORT_SEQ_ID":"1000101","ORIGIN":"ABE","DEST_AIRPORT_SEQ_ID":"1000101","DEST":"ATL","CRS_DEP_TIME":"1600","DEP_TIME":"1555","DEP_DELAY":-5,"TAXI_OUT":7,"WHEELS_OFF":"1602","WHEELS_ON":"1747","TAXI_IN":4,"CRS_ARR_TIME":"1809","ARR_TIME":"1751","ARR_DELAY":-18,"CANCELLED":false,"DIVERTED":false,"DISTANCE":"692.00"}
{'1000101': ('58.10944444', '-152.90666667', 'America/Anchorage')}
{"FL_DATE": "2015-04-28", "UNIQUE_CARRIER": "EV", "ORIGIN_AIRPORT_SEQ_ID": "1000101", "ORIGIN": "ABE", "DEST_AIRPORT_SEQ_ID": "1000101", "DEST": "ATL", "CRS_DEP_TIME": "2015-04-29 00:00:00", "DEP_TIME": "2015-04-28 23:55:00", "DEP_DELAY": -5, "TAXI_OUT": 7, "WHEELS_OFF": "2015-04-29 00:02:00", "WHEELS_ON": "2015-04-29 01:47:00", "TAXI_IN": 4, "CRS_ARR_TIME": "2015-04-29 02:09:00", "ARR_TIME": "2015-04-29 01:51:00", "ARR_DELAY": -18, "CANCELLED": false, "DIVERTED": false, "DISTANCE": "692.00"}


In [ ]:
df.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,W,54,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,W,4,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,W,10,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN


In [ ]:
from timezonefinder import TimezoneFinder
import pandas as pd

my_func = TimezoneFinder().timezone_at  #Note the no parenthesis on the function call!
df = pd.DataFrame({'latitude': [-22.540556,-22.950556,-22.967778], 'longitude': [-43.149167,-43.230833,-43.234444], 'timezone': [0,0,0]})
df['timezone'] = df.apply(lambda x: my_func(lng=x['longitude'], lat=x['latitude']),axis=1)
df

,latitude,longitude,timezone
0,-22.540556,-43.149167,America/Sao_Paulo
1,-22.950556,-43.230833,America/Sao_Paulo
2,-22.967778,-43.234444,America/Sao_Paulo


In [ ]:
from timezonefinder import TimezoneFinder
import pandas as pd

my_func = TimezoneFinder().timezone_at  #Note the no parenthesis on the function call!
df = pd.DataFrame({'latitude': [-22.540556,-22.950556,-22.967778], 'longitude': [-43.149167,-43.230833,-43.234444], 'timezone': [0,0,0]})
df['timezone'] = df.apply(lambda x: my_func(lng=x['longitude'], lat=x['latitude']),axis=1)
df

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824010 sha256=e85a7013765801fc94019167440243eb02fff4f2c180f7af391f603e8180e8b8
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
df.isna().sum()

latitude     0
longitude    0
timezone     0
dtype: int64

In [ ]:
df['LATITUDE'].fillna(1, inplace=True)
df['LONGITUDE'].fillna(1, inplace=True)

In [ ]:
import pandas as pd
df = pd.read_csv('airports.csv',low_memory=False,nrows=3)
df.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,W,54,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,W,4,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,W,10,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN


In [58]:
from timezonefinder import TimezoneFinder
import pandas as pd

my_func = TimezoneFinder().timezone_at  #Note the no parenthesis on the function call!

df['TIMEZONE'] = df.apply(lambda x: my_func(lng=x['LONGITUDE'], lat=x['LATITUDE']),axis=1)
df

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32,TIMEZONE
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,54,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,4,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,10,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN,America/Nome


In [116]:

def as_utc(date, hhmm, tzone):
    try:
        print(date)
        print(hhmm)
        print(tzone)
        if len(hhmm) > 0 and tzone is not None:
            import datetime, pytz
            loc_tz = pytz.timezone(tzone)
            loc_dt = loc_tz.localize(datetime.datetime.strptime(date, '%Y-%m-%d'), is_dst=False)
            # can't just parse hhmm because the data contains 2400 and the like ...
            loc_dt += datetime.timedelta(hours=int(hhmm[:2]), minutes=int(hhmm[2:]))
            utc_dt = loc_dt.astimezone(pytz.utc)
            return utc_dt.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return ''  # empty string corresponds to canceled flights
    except ValueError as e:
        logging.exception('{} {} {}'.format(date, hhmm, tzone))
        raise e

In [50]:
flight = pd.read_json('flights_sample.json',lines=True)
flight

,FL_DATE,UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_SEQ_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DISTANCE
0,2015-04-28,EV,1000101,ABE,1000101,ATL,1600,1555,-5,7,1602,1747,4,1809,1751,-18,False,False,692
1,2015-11-05,DL,1013503,ABE,1039705,ATL,600,556,-4,12,608,749,10,818,759,-19,False,False,692


In [109]:
import pandas as pd
left = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
   'subject_ids':['sub1','sub2','sub4','sub6','sub5']})
right = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
   'subject_id':['sub2','sub4','sub3','sub6','sub5']})
joidf=pd.merge(left, right, how='inner', left_on = 'subject_ids', right_on = 'subject_id')
joidf["id_x"]

0    2
1    3
2    4
3    5
Name: id_x, dtype: int64

In [100]:
df.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32,TIMEZONE,CRS_DEP_TIME
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage,None
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage,None
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN,America/Nome,None


In [51]:
 flight.head()

,FL_DATE,UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_SEQ_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DISTANCE
0,2015-04-28,EV,1000101,ABE,1000101,ATL,1600,1555,-5,7,1602,1747,4,1809,1751,-18,False,False,692
1,2015-11-05,DL,1013503,ABE,1039705,ATL,600,556,-4,12,608,749,10,818,759,-19,False,False,692


In [98]:

print(pd.merge(df,flight, left_on='AIRPORT_SEQ_ID', right_on='ORIGIN_AIRPORT_SEQ_ID'))


   AIRPORT_SEQ_ID  AIRPORT_ID AIRPORT  DISPLAY_AIRPORT_NAME  \
0         1000101       10001     01A  Afognak Lake Airport   

  DISPLAY_AIRPORT_CITY_NAME_FULL  AIRPORT_WAC_SEQ_ID2  AIRPORT_WAC  \
0               Afognak Lake, AK                  101            1   

  AIRPORT_COUNTRY_NAME AIRPORT_COUNTRY_CODE_ISO AIRPORT_STATE_NAME  ...  \
0        United States                       US             Alaska  ...   

  TAXI_OUT  WHEELS_OFF  WHEELS_ON  TAXI_IN CRS_ARR_TIME  ARR_TIME  ARR_DELAY  \
0        7        1602       1747        4         1809      1751        -18   

   CANCELLED DIVERTED  DISTANCE  
0      False    False       692  

[1 rows x 54 columns]


In [102]:
df["CRS_DEP_TIME"]

0    None
1    None
2    None
Name: CRS_DEP_TIME, dtype: object

In [103]:
flight["CRS_DEP_TIME"]

0    1600
1     600
Name: CRS_DEP_TIME, dtype: int64

In [112]:

joindf["CRS_DEP_TIME_y"][0]

1600

In [117]:

joindf["CRS_DEP_TIME"]=joindf.apply(lambda x: as_utc(joindf['FL_DATE'][0],str(joindf['CRS_DEP_TIME_y'][0]),joindf[ "TIMEZONE"][0]),axis=1)


2015-04-28
1600
America/Anchorage


In [118]:
joindf["CRS_DEP_TIME"]

0    2015-04-29 00:00:00
Name: CRS_DEP_TIME, dtype: object

In [124]:
joindf['DEP_TIME']=joindf.apply(lambda x: as_utc(joindf['FL_DATE'][0],str(joindf['CRS_DEP_TIME_y'][0]),joindf[ "TIMEZONE"][0]),axis=1)
joindf['CRS_ARR_TIME']=joindf.apply(lambda x: as_utc(joindf['FL_DATE'][0],str(joindf['CRS_ARR_TIME'][0]),joindf[ "TIMEZONE"][0]),axis=1)
joindf['ARR_TIME']=joindf.apply(lambda x: as_utc(joindf['FL_DATE'][0],str(joindf['ARR_TIME'][0]),joindf[ "TIMEZONE"][0]),axis=1)
joindf['WHEELS_OFF']=joindf.apply(lambda x: as_utc(joindf['FL_DATE'][0],str(joindf['WHEELS_OFF'][0]),joindf[ "TIMEZONE"][0]),axis=1)
joindf['WHEELS_ON']=joindf.apply(lambda x: as_utc(joindf['FL_DATE'][0],str(joindf['WHEELS_ON'][0]),joindf[ "TIMEZONE"][0]),axis=1)


2015-04-28
1600
America/Anchorage
2015-04-28
1809
America/Anchorage
2015-04-28
1751
America/Anchorage
2015-04-28
1602
America/Anchorage
2015-04-28
1747
America/Anchorage


In [125]:
joindf

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DISTANCE,CRS_DEP_TIME
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,2015-04-29 00:02:00,2015-04-29 01:47:00,4,2015-04-29 02:09:00,2015-04-29 01:51:00,-18,False,False,692,2015-04-29 00:00:00
